<a href="https://colab.research.google.com/github/diwashsapkota/Compiler/blob/main/compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Source Here](https://www.twilio.com/blog/abstract-syntax-trees)

Abstract Syntax Trees or ASTs are tree representations of code. They are a fundamental part of the way a compiler works. When a compiler transforms some code there are fundamentally the following steps:
*   Lexical Analysis
*   Syntax Analysis
*   Code Generation


**Lexical Analysis**

During this step, code that you wrote is going to be converted into a set of tokens describing the different parts of your code. This is fundamentally the same method that basic syntax highlighting is using. Tokens don't understand how things fit together and purely focuses on components of a file. You can imagine it as a list or array of different types of tokens.

You can imagine this as if you'd take a text and you break it into words. You might be able to make a differentiation between punctuation, verbs, nouns, numbers, etc. but at this stage you don't have any deeper understanding of what's part of a sentence or how sentences fit together.


**Syntax Analysis aka Parsing**

This is the step where we turn our list of tokens into an Abstract Syntax Tree. It converts our tokens into a tree that represents the actual structure of the code. Where previously in the tokens we only had a pair of () we now have an idea of whether it's a function call, a function definition, a grouping or something else.








In [1]:
import copy #for deepcopy operation
import re #for regular expression related operations

In [21]:
#definition of tokenizer
def tokenizer(input_expression):
  current = 0
  tokens = []

  #initial search variables to help us create tokenizer

  alphabet = re.compile(r"[a-z]", re.I);
  numbers = re.compile(r"[0-9]");
  whiteSpace = (r"\s");

  while current < len(input_expression):
    char = input_expression[current]
    if re.match(whiteSpace, char): #if it is a whitespace, we will continue iterating
      current = current +1
      continue

    if char == '(':
      tokens.append({
          'type': 'left_paren',
          'value': '('
      })
      current = current + 1
      continue
    
    if char == ')':
        token.append({
            'type': 'right_paren',
            'value': ')'
        })
        current = current + 1
        continue
    
    if re.match(numbers, char):
        value = ''
        while re.match(numbers, char):
            value += char
            current = current +1
            char = input_expression[current]
        tokens.append({
            'type': 'number',
            'value': value
        })
        continue

    if re.match(alphabet, char):
        value = ''
        while re.match(alphabet, char):
            value += char
            current = current + 1
            char = input_expression[current]
        tokens.append({
            'type': 'name',
            'value': value
        })
        continue
    raise ValueError("what are THOSE!?!?!" + char);
    return tokens

In [12]:
#definition of a parser function, which will be parsing through the input expression and converting it into tokens
#The tokens are then converted into an abstract syntax tree
def parser(tokens):
    global current
    current = 0
    def walk(): #the nested walk function allows us to use it recursively to build the tree
        global current
        token = tokens[current]
        #the tree will contain bunch of all the nodes we will have
        if token.get('type') == 'number':
            current = current + 1
            return {
                'type': 'NumberLiteral',
                'value': token.get('value')
            }
        if token.get('type') == 'left_paren': #if it is a left parenthesis, we are at the beginning of a statement
            current = current + 1
            token = tokens[current]
            node = {
                'type': 'CallExpression',
                'name': token.get('value'),
                'params': []
            }

            current = current + 1
            token = tokens[current]
            while token.get('type') != 'right_paren':
                node['params'].append(walk()); #this is the core part of the function; the nested recursive function
                token = tokens[current]
            current = current + 1
            return node
        
        raise TypeError(token.get('type'))
    ast = {
        'type': 'Program',
        'body': []
    }

    while current > len(tokens):
        ast['body'].append(walk())
    return ast

In [2]:
#The function below helps to transform the abstract syntax tree constructed using the code above
def traverser(ast, visitor):
    def traverseArray(array, parent):
        for child in array:
            traverseNode(child, parent)
        
    def traverseNode(node, parent):
        #inside the traverseNode, we want to insert an insert to ast
        method = visitor.get(node['type'])
        if method:
            method(node, parent) #ensures we start with the head node at the first
       
       #the way we traverse each node depends upon the type of parameter in the node
        if node['type'] == 'Program':
            traverseArray(node['body'], node)
        elif node['type']=='CallExpression':
            traverseArray(node['params'], node)
        elif node['type'] == 'NumberLiteral':
            0
        else:
            raise TypeError(node['type'])
    traverseNode(ast, None)


In [14]:
#  *An Abstract Syntax Tree, or AST for short, is a deeply nested object that
#  *    represents code in a way that is both easy to work with and tells us a lot
#  *    of information.
#  *
#  * For the following syntax:
#  *
#  *   (add 2 (subtract 4 2))
#  *
#  * Tokens might look something like this:
#  *
#  *   [
#  *     { type: 'paren',  value: '('        },
#  *     { type: 'name',   value: 'add'      },
#  *     { type: 'number', value: '2'        },
#  *     { type: 'paren',  value: '('        },
#  *     { type: 'name',   value: 'subtract' },
#  *     { type: 'number', value: '4'        },
#  *     { type: 'number', value: '2'        },
#  *     { type: 'paren',  value: ')'        },
#  *     { type: 'paren',  value: ')'        },
#  *   ]
#  *
#  * And an Abstract Syntax Tree (AST) might look like this:
#  *
#  *   {
#  *     type: 'Program',
#  *     body: [{
#  *       type: 'CallExpression',
#  *       name: 'add',
#  *       params: [{
#  *         type: 'NumberLiteral',
#  *         value: '2',
#  *       }, {
#  *         type: 'CallExpression',
#  *         name: 'subtract',
#  *         params: [{
#  *           type: 'NumberLiteral',
#  *           value: '4',
#  *         }, {
#  *           type: 'NumberLiteral',
#  *           value: '2',
#  *         }]
#  *       }]
#  *     }]
#  *   }
#  *

In [15]:
def transformer(ast):
    newAst = {
        'type': 'Program',
        'body': []
    }

    oldAst = ast
    ast = copy.deepcopy(oldAst)
    ast['_context'] = newAst.get('body')

    #Since there is no built-in traversal constructs in python, we have to write them separately; one for the call expression and one for the number literal. Js has built-in traversal constructs though
    def CallExpressionTraverse(node, parent):
        expression = { #first create a call expression node
            'type': 'CallExpression',
            'callee': {
                'type': 'Identifier',
                'name': node['name'] #the name of new type callee is same as that of its parent name
            }
            'arguments': []
        }

        node['_context'] = expression['arguments']

        if parent['type'] != 'CallExpression':
            expression = {
                'type': 'ExpressionStatement',
                'expression': expression
            }
        parent['_context'].append(expression)

    def NumberLiteralTraverse(node, parent):
        parent['_context'].append({
            'type': 'NumberLiteral',
            'value': node['value']
        })
    traverser(ast, {
        'NumberLiteral': NumberLiteralTraverse,
        'CallExpression': CallExpressionTraverse
    })

    return newAst

In [3]:
##last part! Code generation
#a recursive stringify function that iterates
#through the newly created AST, node by node, continually
#building a string output given the values in each node.
def codeGenerator(node):
    if node['type'] == 'Program':
        return '\n'.join([code for code in map(codeGenerator, node['body'])])
    elif node['type'] == 'Identifier':
        return node['name']
    elif node['type'] == 'NumberLiteral':
        return node['value']
    elif node['type'] == 'ExpressionStatement':
        expression = codeGenerator(node['expression']) 
        return '%s;' % expression
    elif node['type'] == 'CallExpression':
        callee = codeGenerator(node['callee']) 
        params = ', '.join([code for code in map(codeGenerator, node['arguments'])])
        return "%s(%s)" % (callee, params)
    else:
        raise TypeError(node['type'])

In [18]:
#definition of the working of entire compiler

def compiler (input_expression):
  tokens = tokenizer(input_expression)
  ast = parser(tokens) #ast = Abstract Syntax Tree
  newAst = transformer(ast)
  output = CodeGenerator(newAst)

  return output

In [35]:
def main():
    #test 
    input = "(add 2 (subtract 4 2))"
    output = compiler(input)
    print(output)


if __name__ == "__main__":
    main()

add(2, subtract(4, 2));
